In [ ]:
class foo():
    def __init__(self, value, name):
        self.value = value
        self.name = name
    def __add__(self, other):
        """add values and concatenate names"""
        return foo(self.value + other.value, self.name + other.name)
    def __sub__(self, other):
        """sub values and concatenate names"""
        return foo(self.value - other.value, self.name + other.name)  
    def __repr__(self):
        return f"toto: {self.value}, {self.name}"
    def __array_ufunc__(self, *args, **kwargs):
        print(args)
        print(kwargs)

In [ ]:
a = foo(5, "1st")  
b = foo(1, "2nd")

In [1]:
import numpy as np
import logging
import inspect

In [2]:
HANDLED_FUNCTIONS = {}

In [3]:
def implements(numpy_function):
    """Register an __array_function__ implementation for MyArray objects."""
    def decorator(func):
        HANDLED_FUNCTIONS[numpy_function] = func
        return func
    return decorator

In [4]:
class MyArray(object):
    def __array_function__(self, func, types, args, kwargs):
        logging.debug('{} {}'.format(inspect.currentframe().f_code.co_name, func))
        if func not in HANDLED_FUNCTIONS:
            return NotImplemented
        if not all(issubclass(t, MyArray) for t in types):
            return NotImplemented
        return HANDLED_FUNCTIONS[func](*args, **kwargs)
    
    def __array_ufunc__(self, ufunc, method, inputs, *args, **kwargs):
        logging.debug('{} {}'.format(inspect.currentframe().f_code.co_name, ufunc))
        if ufunc not in HANDLED_FUNCTIONS:
            return NotImplemented
        out = kwargs.pop('out', None)
    
        if out is not None:
            HANDLED_FUNCTIONS[ufunc](inputs, *args, out=out[0], **kwargs)
            return
        else:
            return HANDLED_FUNCTIONS[ufunc](inputs, *args, out=None, **kwargs)
        
    def __init__(self, inlist):
        self.list = inlist[:]        
    
    @property
    def ndim(self):
        return 1

    @property
    def shape(self):
        return (len(self.list), )
    
    @property
    def dtype(self):
        return np.dtype(np.int32)

    def __str__(self):
        return "MyArray " + str(self.list)
    
    def __add__(self, other, *args, **kwargs):
        logging.debug('{}'.format(inspect.currentframe().f_code.co_name))
        return self.add(other, *args, **kwargs)

    @implements(np.add)
    def add(self, *args, **kwargs):
        strng = "MyClass add, out {} {}".format( kwargs.get('out', None), len(args) )
        logging.debug('{} {}'.format(inspect.currentframe().f_code.co_name, strng))
        out = kwargs.get('out', None)
        if out is None:
            return MyArray([el + args[0] for el in self.list])
        else:
            for i,el in enumerate(self.list):
                out[i] = args[0] + el

    # implements np.sum is required when one wants to use the np.sum on this object            
    @implements(np.sum)
    def sum(self, *args, **kwargs):
        return sum(self.list)  # return self.list.ndim    

In [5]:
def main():
    logging.basicConfig(level=logging.DEBUG)

    A = MyArray(np.array([1,2]))
    
    # test np.sum
    print ("sum" , np.sum(A, axis=1))
    
    # test add
    B = A.add(2)
    print(B, 'B')
    
    out = MyArray([20,30])
    print(out,'out')
    A.add(2,out=out)
    print(out,'out')

    # test np.add
    # see comments on __add__ 
    #B = A+2
    B = np.add(A,2)
    print(B, 'B')

    B = A+2
    print(B, 'B')

    np.add(A,2,out=out)
    print(out, "out")

In [6]:
main()

DEBUG:root:__array_function__ <function sum at 0x7f7158266660>
DEBUG:root:add MyClass add, out None 1
DEBUG:root:add MyClass add, out MyArray [20, 30] 1


sum 3
MyArray [np.int64(3), np.int64(4)] B
MyArray [20, 30] out


TypeError: 'MyArray' object does not support item assignment

In [8]:
for t in types:
    print(t)

NameError: name 'types' is not defined